## 环境配置

In [ ]:
!pip install torchtext
!pip install jieba
!pip install sacrebleu

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.nn.utils import clip_grad_norm_
from torchtext.data.metrics import bleu_score
from torch.utils.data import Dataset, DataLoader
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from typing import List, Tuple
import jieba
import random
from torch.nn.utils.rnn import pad_sequence
import sacrebleu
import time
import math

安装spacy用于英文的tokenizer，不同环境的安装请参考：https://spacy.io/usage
en_core_web_sm是用于英文的语言包，由于命令`!python -m spacy download en_core_web_sm`安装有点慢，这里可以离线安装：[https://github.com/explosion/spacy-models/releases](https://github.com/explosion/spacy-models/releases)，下载压缩包后，上传到dataset目录，然后使用`!pip install ../dataset/en_core_web_sm`，en_core_web_sm 3.7.3版本的要求 spaCy >=3.7.2,<3.8.0


In [ ]:
!pip install -U pip setuptools wheel -i https://pypi.tuna.tsinghua.edu.cn/simple
!pip install -U 'spacy[cuda12x]' -i https://pypi.tuna.tsinghua.edu.cn/simple
!pip install ../dataset/en_core_web_trf-3.7.3-py3-none-any.whl
# !python -m spacy download en_core_web_sm

## 数据预处理

In [ ]:
# 定义tokenizer
en_tokenizer = get_tokenizer('spacy', language='en_core_web_trf')
zh_tokenizer = lambda x: list(jieba.cut(x))  # 使用jieba分词

/home/xuhu/miniconda3/envs/glm4/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# 读取数据函数  读取指定路径的文件，并将文件的每一行读取为一个字符串列表。
def read_data(file_path: str) -> List[str]:
    with open(file_path, 'r', encoding='utf-8') as f:  
        return [line.strip() for line in f]   # 使用列表推导式，读取文件的每一行，并去除行首和行尾的空白字符。

# 数据预处理函数  对英文和中文文本进行预处理，包括分词、截断、转换为小写，并返回处理后的文本对。
def preprocess_data(en_data: List[str], zh_data: List[str]) -> List[Tuple[List[str], List[str]]]:
    processed_data = []
    for en, zh in zip(en_data, zh_data):
        en_tokens = en_tokenizer(en.lower())[:MAX_LENGTH]  # 将英文文本转换为小写，并进行分词，截断到最大长度。
        zh_tokens = zh_tokenizer(zh)[:MAX_LENGTH]  # 对中文文本进行分词，截断到最大长度。
        if en_tokens and zh_tokens:  # 确保两个序列都不为空
            processed_data.append((en_tokens, zh_tokens))  # 将处理后的文本对添加到列表中。
    return processed_data  # 返回处理后的文本对列表。

# 构建词汇表  根据预处理后的数据构建英文和中文的词汇表
def build_vocab(data: List[Tuple[List[str], List[str]]]):
    en_vocab = build_vocab_from_iterator(
        (en for en, _ in data),
        specials=['<unk>', '<pad>', '<bos>', '<eos>']
    )  # 从英文数据生成词汇表，并添加特殊标记。
    zh_vocab = build_vocab_from_iterator(
        (zh for _, zh in data),
        specials=['<unk>', '<pad>', '<bos>', '<eos>']
    )   # 从中文数据生成词汇表，并添加特殊标记。
    en_vocab.set_default_index(en_vocab['<unk>'])  # 设置默认索引为<unk>
    zh_vocab.set_default_index(zh_vocab['<unk>'])  # 设置默认索引为<unk>
    return en_vocab, zh_vocab  # 返回英文和中文的词汇表。



In [ ]:

class TranslationDataset(Dataset):
    def __init__(self, data: List[Tuple[List[str], List[str]]], en_vocab, zh_vocab):
        self.data = data  # data 参数是一个包含多个元组的列表，每个元组包含两个列表：一个英文句子列表和一个中文句子列表。
        self.en_vocab = en_vocab  # en_vocab 参数是一个字典，键是英文单词，值是对应的索引。
        self.zh_vocab = zh_vocab  # zh_vocab 参数是一个字典，键是中文单词，值是对应的索引。

    def __len__(self):
        return len(self.data)  # 返回数据集的长度，即数据集中包含的句子对的数量。

    def __getitem__(self, idx):  # 用于获取数据集中的单个样本，并将其转换为模型可接受的格式。
        en, zh = self.data[idx]  # 获取指定索引的英文和中文句子。
        # 将英文句子和中文句子转换为模型可接受的格式。
        # 每个句子的索引列表以 <bos>（句子开始标记）开始，以 <eos>（句子结束标记）结束。
        en_indices = [self.en_vocab['<bos>']] + [self.en_vocab[token] for token in en] + [self.en_vocab['<eos>']]
        zh_indices = [self.zh_vocab['<bos>']] + [self.zh_vocab[token] for token in zh] + [self.zh_vocab['<eos>']]
        return en_indices, zh_indices  # 返回英文句子和中文句子的索引列表。


In [ ]:
def collate_fn(batch):
    en_batch, zh_batch = [], []  # 定义两个空列表，用于存储英文和中文的序列。
    for en_item, zh_item in batch:
        if en_item and zh_item:  # 确保两个序列都不为空
            # print("都不为空")
            en_batch.append(torch.tensor(en_item))  # 将英文序列转换为张量，并添加到列表中。
            zh_batch.append(torch.tensor(zh_item))  # 将中文序列转换为张量，并添加到列表中。
        else:
            print("存在为空")
    if not en_batch or not zh_batch:  # 如果整个批次为空，返回空张量
        return torch.tensor([]), torch.tensor([])
    
    # src_sequences = [item[0] for item in batch]
    # trg_sequences = [item[1] for item in batch]
    
    # 使用 nn.utils.rnn.pad_sequence 函数对 en_batch 和 zh_batch 进行填充，使它们具有相同的长度。
    # 填充值为各自词汇表中的 <pad> 标记。
    en_batch = nn.utils.rnn.pad_sequence(en_batch, batch_first=True, padding_value=en_vocab['<pad>'])
    zh_batch = nn.utils.rnn.pad_sequence(zh_batch, batch_first=True, padding_value=zh_vocab['<pad>'])

    # en_batch = pad_sequence(en_batch, batch_first=True, padding_value=en_vocab['<pad>'])
    # zh_batch = pad_sequence(zh_batch, batch_first=True, padding_value=zh_vocab['<pad>'])
    
    return en_batch, zh_batch  # 返回填充后的英文和中文序列。

In [ ]:
# 数据加载函数  准备训练集、开发集和测试集的数据以及构建相应的数据加载器。
def load_data(train_path: str, dev_en_path: str, dev_zh_path: str, test_en_path: str):
    # 读取训练数据
    train_data = read_data(train_path)
    train_en, train_zh = zip(*(line.split('\t') for line in train_data))
    # zip函数用于将多个可迭代对象中对应的元素打包成元组，然后返回这些元组组成的对象。
    # 在这个代码中，zip函数结合了生成器表达式，首先对train_data中的每一行进行split('\t')操作，然后将分割后的英文句子和中文句子分别存储在train_en和train_zh中。
    # 生成器表达式*(line.split('\t') for line in train_data)会生成一个迭代器，其中的每个元素都是train_data中的一行以制表符分隔后的结果。
    # zip函数会同时遍历这些迭代器，将对应位置的元素打包成元组，最终生成包含了所有对应元素的元组的zip对象。
    # 然后使用*将zip对象解压缩并一一对应地赋值给train_en和train_zh。
    
    # 读取开发集和测试集
    dev_en = read_data(dev_en_path)
    dev_zh = read_data(dev_zh_path)
    test_en = read_data(test_en_path)

    # 预处理数据
    train_processed = preprocess_data(train_en, train_zh)
    dev_processed = preprocess_data(dev_en, dev_zh)
    test_processed = [(en_tokenizer(en.lower())[:MAX_LENGTH], []) for en in test_en if en.strip()]
    # 最后一行是一个列表推导式，其目的是对测试集中的英文句子进行预处理，并构建一个包含元组的列表。
    # 在元组中，第一个元素是经过en_tokenizer处理后的英文句子，并且将其转换为小写形式，然后截取前MAX_LENGTH个词语；第二个元素是一个空列表。

    # 构建词汇表
    global en_vocab, zh_vocab  # 声明全局变量，在函数内部对这两个变量的修改将影响到其在函数外部的值
    en_vocab, zh_vocab = build_vocab(train_processed)

    # 创建数据集
    train_dataset = TranslationDataset(train_processed, en_vocab, zh_vocab)
    dev_dataset = TranslationDataset(dev_processed, en_vocab, zh_vocab)
    test_dataset = TranslationDataset(test_processed, en_vocab, zh_vocab)
    
    from torch.utils.data import Subset
    # Subset 类的作用： Subset 类是 PyTorch 提供的一个工具类，用于从现有的数据集中选择一个子集。
    # 这在数据集过大或者只需要使用部分数据进行训练或测试时非常有用。

    # 只用前N个样本进行测试
    indices = list(range(N))
    train_dataset = Subset(train_dataset, indices)
    # indices 是一个包含前 N 个索引的列表，train_dataset 是原始的训练数据集。
    # 通过 Subset(train_dataset, indices)，我们创建了一个新的数据集 train_dataset，它只包含原始数据集中的前 N 个样本。

    # 创建数据加载器
    # train_dataset: 这是已经预处理过的训练数据集。
    # batch_size=BATCH_SIZE: 每个批次的大小，BATCH_SIZE 是一个预定义的常量，表示每个批次包含的样本数量。
    # shuffle=True: 在每个 epoch 开始前打乱数据，以确保模型在不同 epoch 中看到的数据顺序不同，有助于模型更好地学习。
    # collate_fn=collate_fn: 这是一个自定义的函数，用于将一个 batch 的数据进行整理，使其适合模型输入。
    # drop_last=True: 如果最后一个 batch 的大小小于 batch_size，则丢弃这个 batch，以确保所有 batch 的大小一致。
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn, drop_last=True)  # 训练集数据加载器
    dev_loader = DataLoader(dev_dataset, batch_size=BATCH_SIZE, collate_fn=collate_fn, drop_last=True)  # 开发集数据加载器
    test_loader = DataLoader(test_dataset, batch_size=1, collate_fn=collate_fn, drop_last=True)  # 测试集数据加载器

    return train_loader, dev_loader, test_loader, en_vocab, zh_vocab

In [ ]:
# def pad_sequence(batch,batch_first,padding_value):
#     # 对源语言和目标语言序列分别进行填充
#     src_sequences = [item[0] for item in batch]
#     trg_sequences = [item[1] for item in batch]
    
#     src_padded = nn.utils.rnn.pad_sequence(src_sequences, batch_first=True, padding_value=en_vocab['<pad>'])
#     trg_padded = nn.utils.rnn.pad_sequence(trg_sequences, batch_first=True, padding_value=zh_vocab['<pad>'])
    
#     return src_padded, trg_padded



## 模型构建

In [ ]:

class Encoder(nn.Module):
    # 输入词汇的维度大小，嵌入层维度大小，隐藏层维度大小，层数，dropout的概率
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        self.hid_dim = hid_dim  
        self.n_layers = n_layers  
        
        self.embedding = nn.Embedding(input_dim, emb_dim)  # 词嵌入层
        self.gru = nn.GRU(emb_dim, hid_dim, n_layers, dropout=dropout, batch_first=True)  # 编码器GRU层
        self.dropout = nn.Dropout(dropout)  # 随机失活层
        
    def forward(self, src):
        # src = [batch size, src len]
        embedded = self.dropout(self.embedding(src))  # 输入 src 经过嵌入层和 Dropout 层，得到 embedded。
        # embedded = [batch size, src len, emb dim]
        
        outputs, hidden = self.gru(embedded)  # 输入 embedded 经过编码器GRU层，得到 outputs 和 hidden。
        # outputs = [batch size, src len, hid dim * n directions]
        # hidden = [n layers * n directions, batch size, hid dim]
        
        return outputs, hidden

class Attention(nn.Module):
    def __init__(self, hid_dim):
        super().__init__()
        # 定义注意力层，输入为编码器的隐藏层和解码器的隐藏层，输出为注意力权重。
        self.attn = nn.Linear(hid_dim * 2, hid_dim)
        self.v = nn.Linear(hid_dim, 1, bias=False)
        
    def forward(self, hidden, encoder_outputs):
        # hidden = [1, batch size, hid dim]
        # encoder_outputs = [batch size, src len, hid dim]
        
        batch_size = encoder_outputs.shape[0]
        src_len = encoder_outputs.shape[1]
        
        hidden = hidden.repeat(src_len, 1, 1).transpose(0, 1)
        # hidden = [batch size, src len, hid dim]
        # 重复 hidden 【src_len】 次， 使其形状与 encoder_outputs 相同，然后与 encoder_outputs 拼接。
        
        # 拼接后的结果经过 attn 层和 tanh 激活函数，得到 energy。
        energy = torch.tanh(self.attn(torch.cat((hidden, encoder_outputs), dim=2)))
        # energy = [batch size, src len, hid dim]
        
        # energy 经过 v 层，得到 attention。
        attention = self.v(energy).squeeze(2)
        # attention = [batch size, src len]
        
        # attention 经过 softmax 函数，得到最终的注意力权重。
        return F.softmax(attention, dim=1)

class Decoder(nn.Module):
    # 输出词汇的维度大小，嵌入层维度大小，隐藏层维度大小，层数，dropout的概率，注意力层
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout, attention):
        super().__init__()
        self.output_dim = output_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        self.attention = attention
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        self.gru = nn.GRU(hid_dim + emb_dim, hid_dim, n_layers, dropout=dropout, batch_first=True)
        self.fc_out = nn.Linear(hid_dim * 2 + emb_dim, output_dim)  # 全连接层 输出层
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, encoder_outputs):
        # input = [batch size, 1]
        # hidden = [n layers, batch size, hid dim]
        # encoder_outputs = [batch size, src len, hid dim]
        
        input = input.unsqueeze(1)
        embedded = self.dropout(self.embedding(input))  # 输入 input 经过嵌入层和 Dropout 层，得到 embedded。
        # embedded = [batch size, 1, emb dim]
        
        a = self.attention(hidden[-1:], encoder_outputs)  # hidden 和 encoder_outputs 经过注意力层，得到注意力权重 a。
        # a = [batch size, src len]
        
        a = a.unsqueeze(1)
        # a = [batch size, 1, src len]
        
        weighted = torch.bmm(a, encoder_outputs)  # 使用注意力权重 a 对 encoder_outputs 加权，得到 weighted。
        # weighted = [batch size, 1, hid dim]
        
        rnn_input = torch.cat((embedded, weighted), dim=2)  # 将 embedded 和 weighted 按维度 2 拼接，得到 rnn_input。
        # rnn_input = [batch size, 1, emb dim + hid dim]
        
        output, hidden = self.gru(rnn_input, hidden)  # rnn_input 和 hidden 经过解码器GRU层，得到 output 和 hidden。
        # output = [batch size, 1, hid dim]
        # hidden = [n layers, batch size, hid dim]
        
        embedded = embedded.squeeze(1)
        output = output.squeeze(1)
        weighted = weighted.squeeze(1)
        
        prediction = self.fc_out(torch.cat((output, weighted, embedded), dim=1))  # 将 output、weighted 和 embedded 按维度 1 拼接，经过全连接层，得到 prediction。
        # prediction = [batch size, output dim]
        
        return prediction, hidden

class Seq2Seq(nn.Module):
    # 编码器，解码器，设备（cpu或gpu）
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
    def forward(self, src, trg, teacher_forcing_ratio=0.5):  # 输入 src 和 trg，输出模型预测的输出。
        # src = [batch size, src len]
        # trg = [batch size, trg len]
        
        batch_size = src.shape[0]
        trg_len = trg.shape[1]
        trg_vocab_size = self.decoder.output_dim
        
        outputs = torch.zeros(batch_size, trg_len, trg_vocab_size).to(self.device)  # 初始化输出张量。
        encoder_outputs, hidden = self.encoder(src)  # 使用编码器对 src 进行编码，得到 encoder_outputs 和 hidden。
        # encoder_outputs = [batch size, src len, hid dim * n directions]
        # hidden = [n layers * n directions, batch size, hid dim]
        
        input = trg[:, 0]  # 输入 trg 的第一个词。初始化解码器的输入 input。
        # input = [batch size]
        
        for t in range(1, trg_len):  # 循环解码 trg_len - 1 次
            output, hidden = self.decoder(input, hidden, encoder_outputs)  # 使用解码器对 input 和 hidden 进行解码，得到 output 和 hidden。
            # output = [batch size, output dim]
            # hidden = [n layers * n directions, batch size, hid dim]
            outputs[:, t] = output  # 保存 output 到 outputs。
            
            # 决定是否使用教师强制法。
            teacher_force = random.random() < teacher_forcing_ratio
            top1 = output.argmax(1)  # 选择 output 的最大值作为下一个输入词。
            input = trg[:, t] if teacher_force else top1  # 更新 input，决定下一个输入词。
        
        return outputs  

In [ ]:

# 初始化模型
def initialize_model(input_dim, output_dim, emb_dim, hid_dim, n_layers, dropout, device):
    attn = Attention(hid_dim)
    enc = Encoder(input_dim, emb_dim, hid_dim, n_layers, dropout)
    dec = Decoder(output_dim, emb_dim, hid_dim, n_layers, dropout, attn)
    model = Seq2Seq(enc, dec, device).to(device)
    return model


## 训练

In [ ]:
# 定义优化器
def initialize_optimizer(model, learning_rate=0.001):
    return optim.Adam(model.parameters(), lr=learning_rate)  # 使用torch.optim库中的Adam优化器，返回一个优化器对象。

In [ ]:
# 计算运行时间 ： 大概10分钟左右
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
# 这是一个用于训练神经网络模型的训练循环函数。
# iterator 数据迭代器
# optimizer 优化器
# criterion 损失函数
# clip 最大梯度范数
def train(model, iterator, optimizer, criterion, clip):
    model.train()  # 将模型设置为训练模式，这意味着模型将被用来进行前向传播和反向传播来更新参数。
    epoch_loss = 0  # 初始化一个变量epoch_loss，用于累积整个epoch的损失值。
    
    for i, batch in enumerate(iterator):
        #print(f"Training batch {i}")

        src, trg = batch  # 取出一个batch的数据，从中获取输入数据src和目标数据trg。

        #print(f"Source shape before: {src.shape}, Target shape before: {trg.shape}")

        if src.numel() == 0 or trg.numel() == 0:
            #print("Empty batch detected, skipping...")
            continue  # 跳过空的批次
        
        src, trg = src.to(DEVICE), trg.to(DEVICE)  # 将输入数据和目标数据移动到指定的设备上（例如GPU）进行加速计算。
        
        #print(f"Source shape after: {src.shape}, Target shape after: {trg.shape}")
        
        optimizer.zero_grad()  # 每个批次开始时，将优化器中的梯度置零，以准备接收当前批次的损失值。
        output = model(src, trg)  # 通过模型进行前向传播，使用输入数据src来预测目标数据trg。
        
        #print(f"Output shape: {output.shape}")
        
        output_dim = output.shape[-1]  # 获取模型输出的最后一个维度的大小。
        output = output[:, 1:].contiguous().view(-1, output_dim)  
        # 从模型预测输出中去掉输出的开始标记，并将其余输出按行排列，将其变形为一个二维张量，其行数为batch_size*max_len-1，列数为output_dim。
        trg = trg[:, 1:].contiguous().view(-1)  # 将目标数据中的起始标记去掉，并将其变形为一个一维张量。
        
        loss = criterion(output, trg)  # 使用损失函数计算模型预测输出与真实目标之间的损失值。
        loss.backward()  # 通过反向传播计算模型中各参数对损失值的梯度。
        
        clip_grad_norm_(model.parameters(), clip)  # 将模型参数的梯度限制在最大范数clip内，以防止梯度爆炸。
        optimizer.step()  # 使用优化器更新模型参数，使得损失值最小。
        
        epoch_loss += loss.item()  # 累积当前批次的损失值。

    print(f"Average loss for this epoch: {epoch_loss / len(iterator)}")  # 打印当前epoch的平均损失值。
    return epoch_loss / len(iterator)  # 返回当前epoch的平均损失值。

# 这是一个用于评估神经网络模型的评估循环函数。
def evaluate(model, iterator, criterion):
    model.eval()  # 将模型设置为评估模式，这意味着模型将不会更新参数。

    epoch_loss = 0

    # torch.no_grad()：禁用梯度计算，因为在评估阶段不需要进行反向传播，这样可以节省内存并加快计算速度。
    with torch.no_grad():
        for i, batch in enumerate(iterator):
            #print(f"Evaluating batch {i}")
            src, trg = batch
            if src.numel() == 0 or trg.numel() == 0:
                continue  # 跳过空批次
            
            src, trg = src.to(DEVICE), trg.to(DEVICE)
            
            output = model(src, trg, 0)  
            # output = model.forward(src, trg, 0)
            # 通过模型进行前向传播，使用输入数据 src 来预测目标数据 trg。
            # 最后一个参数 0 表示关闭 teacher forcing，即在解码过程中不使用真实的目标数据作为输入。
            
            output_dim = output.shape[-1]
            output = output[:, 1:].contiguous().view(-1, output_dim)
            trg = trg[:, 1:].contiguous().view(-1)
            
            loss = criterion(output, trg)
            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [ ]:
# 翻译函数
# 接受输入句子、源词汇表、目标词汇表、模型、设备和最大长度等参数。
def translate_sentence(sentence, src_vocab, trg_vocab, model, device, max_length=50):
    model.eval()

    #print(sentence)        # 打印sentence的内容

    if isinstance(sentence, str):  # 如果输入的句子是字符串，则先用英文分词器进行分词，然后转化为token列表
        # tokens = [token.lower() for token in en_tokenizer(sentence)]
        tokens = [token for token in en_tokenizer(sentence)]
    else:  # 如果输入的句子是token列表，则直接使用
        #tokens = [token.lower() for token in sentence]
        tokens = [str(token) for token in sentence]
        
    tokens = ['<bos>'] + tokens + ['<eos>']  # 在分词结果的开头添加 <bos>（开始标记），在结尾添加 <eos>（结束标记）。
    
    #print(tokens)        # 打印分词后的token列表

    src_indexes = [src_vocab[token] for token in tokens]  # 将分词结果token转换为源词汇表中的索引。
    src_tensor = torch.LongTensor(src_indexes).unsqueeze(0).to(device)  
    # 将索引列表转换为 PyTorch 张量，并在第0维增加一个维度，然后将张量移动到指定设备。
    
    # 使用编码器对输入句子进行编码，得到编码器输出和隐藏状态。
    with torch.no_grad():  
        encoder_outputs, hidden = model.encoder(src_tensor)  
    
    trg_indexes = [trg_vocab['<bos>']]  # 初始化目标索引列表，添加 <bos> 标记。

    # 在循环中，使用解码器逐个生成目标词汇的索引，直到达到最大长度或生成 <eos> 标记。
    for i in range(max_length):  
        trg_tensor = torch.LongTensor([trg_indexes[-1]]).to(device)
        
        with torch.no_grad():
            output, hidden = model.decoder(trg_tensor, hidden, encoder_outputs)
        
        pred_token = output.argmax(1).item()
        trg_indexes.append(pred_token)
        
        if pred_token == trg_vocab['<eos>']:
            break
    
    trg_tokens = [trg_vocab.get_itos()[i] for i in trg_indexes]  # 将目标索引列表转换为目标词汇列表。
    return trg_tokens[1:-1]  # 移除 <bos> 和 <eos> 并返回。

In [ ]:
# 计算bleu
# 参数：数据加载器、源词汇表、目标词汇表、模型、设备
def calculate_bleu(dev_loader, src_vocab, trg_vocab, model, device):
    translated_sentences = []  # 翻译结果
    references = []  # 参考翻译

    for src, trg in dev_loader:
        src = src.to(device)
        translation = translate_sentence(src, src_vocab, trg_vocab, model, device)
        # 将翻译结果转换为字符串
        translated_sentences.append(' '.join(translation))

        # 将每个参考翻译转换为字符串，并添加到references列表中
        for t in trg:  # 遍历目标句子 trg 中的每个句子。
            # 将每个目标句子转换为字符串，并过滤掉特殊标记 <bos>、<eos> 和 <pad>。
            ref_str = ' '.join([trg_vocab.get_itos()[idx] for idx in t.tolist() if idx not in [trg_vocab['<bos>'], trg_vocab['<eos>'], trg_vocab['<pad>']]])
            references.append(ref_str)

    # print("translated_sentences",translated_sentences[:2])
    # print("references:",references[6:8])
    # 使用`sacrebleu`计算BLEU分数
    # 注意：sacrebleu要求references是一个列表的列表，其中每个子列表包含一个或多个参考翻译
    bleu = sacrebleu.corpus_bleu(translated_sentences, [references])

    # 打印BLEU分数
    print(f"BLEU score: {bleu.score}")

    return bleu.score

In [ ]:
# 主训练循环
def train_model(model, train_iterator, valid_iterator, optimizer, criterion, N_EPOCHS=10, CLIP=1):
    best_valid_loss = float('inf')
    # float('inf')：这是一个特殊的浮点数值，表示正无穷大，表示一个比任何有限数都大的值。
    
    for epoch in range(N_EPOCHS):
        start_time = time.time()
        
        print(f"Starting Epoch {epoch + 1}")
        train_loss = train(model, train_iterator, optimizer, criterion, CLIP)  # 训练的平均损失值
        valid_loss = evaluate(model, valid_iterator, criterion)  # 评估的平均损失值
        
        end_time = time.time()  
        epoch_mins, epoch_secs = epoch_time(start_time, end_time)  
        
        # 当验证损失（valid_loss）小于之前记录的最佳验证损失（best_valid_loss）时，
        # 代码会更新 best_valid_loss 并调用 torch.save 函数，
        # 将当前模型的状态字典保存到 ../model/best-model_test.pt 文件中。
        # 这样做的目的是为了保留在训练过程中表现最好的模型的参数，以便后续可以加载这个模型进行预测或进一步的训练。
        if valid_loss < best_valid_loss:
            best_valid_loss = valid_loss
            torch.save(model.state_dict(), '../model/best-model_test.pt')

        
        print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
        print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
        print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

        ''' train_loss 是训练过程中一个非常重要的指标，它有以下几个主要用途：
            1.监控训练过程：
                train_loss 可以帮助我们监控模型的训练进度和效果。
                通过观察 train_loss 的变化，我们可以了解模型是否在逐渐学习并改进其预测能力。
                如果 train_loss 在每个 epoch 后逐渐下降，这通常表明模型正在学习并改进其性能。
            2.调整学习率：
                在一些训练策略中，学习率可能会根据 train_loss 的变化进行调整。
                例如，如果 train_loss 下降缓慢或停滞不前，可能需要降低学习率以帮助模型更好地收敛。
            3.早停法（Early Stopping）：
                虽然在这个代码示例中没有直接实现早停法，但 train_loss 可以与验证损失（valid_loss）结合使用来实现早停法。早停法是一种防止过拟合的技术，当验证损失不再下降时，可以提前终止训练。
            4.模型选择：
                在某些情况下，train_loss 可以用于选择最佳的模型。
                例如，在多个模型或超参数组合中，可以选择 train_loss 最低的模型作为最终模型。'''

In [ ]:
# 定义常量
MAX_LENGTH = 1000  # 最大句子长度
BATCH_SIZE = 32
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
N = 100   # 采样训练集的数量

train_path = '../dataset/train.txt'
dev_en_path = '../dataset/dev_en.txt'
dev_zh_path = '../dataset/dev_zh.txt'
test_en_path = '../dataset/test_en.txt'

train_loader, dev_loader, test_loader, en_vocab, zh_vocab = load_data(
    train_path, dev_en_path, dev_zh_path, test_en_path
)


print(f"英语词汇表大小: {len(en_vocab)}")
print(f"中文词汇表大小: {len(zh_vocab)}")
print(f"训练集大小: {len(train_loader.dataset)}")
print(f"开发集大小: {len(dev_loader.dataset)}")
print(f"测试集大小: {len(test_loader.dataset)}")

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.821 seconds.
Prefix dict has been built successfully.


英语词汇表大小: 44850
中文词汇表大小: 75172
训练集大小: 100
开发集大小: 1000
测试集大小: 1000


In [ ]:
# 主函数
if __name__ == '__main__':
    
    N_EPOCHS = 10
    CLIP=1
    # 模型参数
    INPUT_DIM = len(en_vocab)
    OUTPUT_DIM = len(zh_vocab)
    EMB_DIM = 128
    HID_DIM = 256
    N_LAYERS = 1
    DROPOUT = 0.5
    
    # 初始化模型
    model = initialize_model(INPUT_DIM, OUTPUT_DIM, EMB_DIM, HID_DIM, N_LAYERS, DROPOUT, DEVICE)
    print(f'The model has {sum(p.numel() for p in model.parameters() if p.requires_grad):,} trainable parameters')

    # 定义损失函数
    criterion = nn.CrossEntropyLoss(ignore_index=zh_vocab['<pad>'])
    # 初始化优化器
    optimizer = initialize_optimizer(model)

    # 训练模型
    train_model(model, train_loader, dev_loader, optimizer, criterion, N_EPOCHS, CLIP)


/home/xuhu/miniconda3/envs/glm4/lib/python3.12/site-packages/torch/nn/modules/rnn.py:83: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


The model has 64,469,156 trainable parameters


## 在开发集上进行评价

In [ ]:
# 加载最佳模型
model.load_state_dict(torch.load('../model/best-model_test.pt'))

# 计算BLEU分数
bleu_score = calculate_bleu(dev_loader, en_vocab, zh_vocab, model, DEVICE)
print(f'BLEU score = {bleu_score*100:.2f}')


translated_sentences ['非常复杂 Bedi 只手 环视 净水 Swami 0% 贴满 there 幸灾乐祸 了 水压 牛角', '非常复杂 Bedi 只手 环视 净水 Swami 0% 贴满 there 幸灾乐祸 了 水压 牛角']
references: ['只是 你们 无法 见到 这位 天才 设计者', '在 印度 ， 那里 有 一对 夫妻 经营 的 流动 图书馆']
BLEU score = 5.88


## 对测试集进行翻译

In [ ]:
# 加载最佳模型
model.load_state_dict(torch.load('../model/best-model_test.pt'))

<All keys matched successfully>

In [ ]:
with open('../results/submit_test.txt', 'w') as f:
    translated_sentences = []
    for batch in test_loader:  # 遍历所有数据
        src, _ = batch
        src = src.to(DEVICE)
        translated = translate_sentence(src[0], en_vocab, zh_vocab, model, DEVICE)  #翻译结果
        results = "".join(translated)
        f.write(results + '\n')  # 将结果写入文件